# Data Visualization with Modern Data Science

> Assignment 3

Yao-Jen Kuo <yaojenkuo@ntu.edu.tw> from [DATAINPOINT](https://www.datainpoint.com)

## Instructions

- It is highly recommended that you test your solution in SQLiteStudio then paste into Google Colab.
- Write down your solution between comments `-- BEGIN SOLUTION` and `-- END SOLUTION`.
- Running tests to see if your solutions are right:
    - Runtime -> Restart and run all.
- When you are ready to submit, click File -> Download -> Download `.py`.

![](https://i.imgur.com/Y1BcDdx.png)

- Open a new Colab in a private window, upload the script and run tests again before submission to make sure the script is executable in a fresh new Colab.

![](https://i.imgur.com/ojlvbds.png)

- Upload to the Assignment session on NTU COOL.

## Run the cell below to download given files at your working directory.

In [1]:
import unittest
import requests
import numpy as np
import pandas as pd
import sqlite3

file_names = ["nba.db"]
for file_name in file_names:
    file_url = f"https://raw.githubusercontent.com/datainpoint/asgmts-data-viz-with-modern-ds-2023/main/{file_name}"
    r = requests.get(file_url)
    with open(file_name , "wb") as f:
        f.write(r.content)

![](https://raw.githubusercontent.com/datainpoint/asgmts-data-viz-with-modern-ds-2023/main/nbadb-erd.png)

## 01. Write a SQL statement that is able to retrieve the table list given database `nba.db`.

Hint: querying metadata table `sqlite_master`.

```
    type              name
0  table  career_summaries
1  table           players
2  table             teams
```

In [2]:
retrieve_table_list_from_nbadb =\
"""
-- BEGIN SOLUTION
SELECT type,
       name
  FROM sqlite_master
 WHERE type = 'table'
 ORDER BY name;
-- END SOLUTION
"""

## 02. Write a SQL statement that is able to retrieve the table information given database `nba.db`.

Hint: number of columns can be queried through `PRAGMA_TABLE_INFO()`.

```
               name  number_of_rows  number_of_columns
0  career_summaries             502                 31
1           players             502                 18
2             teams              30                 12
```

In [3]:
retrieve_table_info_from_nbadb =\
"""
-- BEGIN SOLUTION
SELECT 'career_summaries' AS name,
       COUNT( * ) AS number_of_rows,
       pragma_career_summaries.number_of_columns
  FROM career_summaries
       JOIN
       (
           SELECT 'career_summaries' AS name,
                  COUNT( * ) AS number_of_columns
             FROM pragma_table_info('career_summaries') 
       )
       AS pragma_career_summaries ON name = pragma_career_summaries.name
 UNION
SELECT 'players' AS name,
       COUNT( * ) AS number_of_rows,
       pragma_players.number_of_columns
  FROM players
       JOIN
       (
           SELECT 'players' AS name,
                  COUNT( * ) AS number_of_columns
             FROM pragma_table_info('players') 
       )
       AS pragma_players ON name = pragma_players.name
 UNION
SELECT 'teams' AS name,
       COUNT( * ) AS number_of_rows,
       pragma_teams.number_of_columns
  FROM teams
       JOIN
       (
           SELECT 'teams' AS name,
                  COUNT( * ) AS number_of_columns
             FROM pragma_table_info('teams') 
       )
       AS pragma_teams ON name = pragma_teams.name;
-- END SOLUTION
"""

## 03. Write a SQL statement that is able to count the number of players from each country in `players`.

```
                   country  number_of_players
0                      USA                384
1                   Canada                 20
2                   France                 10
3                  Germany                  8
4                Australia                  7
5                   Serbia                  5
6                    Spain                  5
7                  Nigeria                  4
8                   Turkey                  4
9                  Bahamas                  3
10                 Croatia                  3
11                   Japan                  3
12               Lithuania                  3
13                Slovenia                  3
14               Argentina                  2
15                  Brazil                  2
16                Cameroon                  2
17          Czech Republic                  2
18                     DRC                  2
19      Dominican Republic                  2
20                 Georgia                  2
21                  Greece                  2
22                  Latvia                  2
23              Montenegro                  2
24                 Ukraine                  2
25          United Kingdom                  2
26                  Angola                  1
27                 Austria                  1
28  Bosnia and Herzegovina                  1
29                 Denmark                  1
30                 Finland                  1
31                  Israel                  1
32                   Italy                  1
33                 Jamaica                  1
34             New Zealand                  1
35                Portugal                  1
36   Republic of the Congo                  1
37             Saint Lucia                  1
38                 Senegal                  1
39             South Sudan                  1
40                   Sudan                  1
41             Switzerland                  1
```

In [4]:
count_players_from_each_country =\
"""
-- BEGIN SOLUTION
SELECT country,
       COUNT(*) AS number_of_players
  FROM players
 GROUP BY country
 ORDER BY number_of_players DESC,
          country;
-- END SOLUTION
"""

## 04. Write a SQL statement that is able to count the number of players from USA or not in `players`.

```
  from_usa_or_not  number_of_players
0        From USA                384
1    Not from USA                118
```

In [5]:
count_players_from_usa_or_not =\
"""
-- BEGIN SOLUTION
SELECT CASE WHEN country = 'USA' THEN 'From USA'
            ELSE 'Not from USA' END AS from_usa_or_not,
       COUNT(*) AS number_of_players
  FROM players
 GROUP BY from_usa_or_not;
-- END SOLUTION
"""

## 05. Write a SQL statement that is able to count the number of players for each position in `players`.

```
   pos  number_of_players
0    G                195
1    F                140
2  G-F                 47
3  F-C                 37
4    C                 37
5  C-F                 26
6  F-G                 20
```

In [6]:
count_players_for_each_position =\
"""
-- BEGIN SOLUTION
SELECT pos,
       COUNT(*) AS number_of_players
  FROM players
 GROUP BY pos
 ORDER BY number_of_players DESC;
-- END SOLUTION
"""

## 06. Write a SQL statement that is able to re-categorize the number of players for each position according the first letter of `pos` in `players`.

```
  new_pos  number_of_players
0       G                242
1       F                197
2       C                 63
```

In [7]:
count_players_for_each_new_position =\
"""
-- BEGIN SOLUTION
SELECT CASE WHEN SUBSTR(pos, 1, 1) = 'G' THEN 'G'
            WHEN SUBSTR(pos, 1, 1) = 'F' THEN 'F'
            WHEN SUBSTR(pos, 1, 1) = 'C' THEN 'C' END AS new_pos,
       COUNT(*) AS number_of_players
  FROM players
 GROUP BY new_pos
 ORDER BY number_of_players DESC;
-- END SOLUTION
"""

## 07. Write a SQL statement that is able to calculate the proportion for each position in `players`.

```
   pos proportion
0    C      7.37%
1  C-F      5.18%
2    F     27.89%
3  F-C      7.37%
4  F-G      3.98%
5    G     38.84%
6  G-F      9.36%
```

In [8]:
calculate_proportion_for_each_position =\
"""
-- BEGIN SOLUTION
SELECT pos,
       ROUND(COUNT(*)*100.0 / (SELECT COUNT(*)
                                 FROM players), 2) || '%' AS proportion
  FROM players
 GROUP BY pos;
-- END SOLUTION
"""

## 08. Write a SQL statement that is able to find the roster of Phoenix Suns in `teams` and `players`.

```
        teamName  pos firstName   lastName
0   Phoenix Suns    C   Bismack    Biyombo
1   Phoenix Suns    C   Deandre      Ayton
2   Phoenix Suns  C-F    JaVale      McGee
3   Phoenix Suns    F       Jae    Crowder
4   Phoenix Suns    F    Torrey      Craig
5   Phoenix Suns    F     Mikal    Bridges
6   Phoenix Suns    F   Cameron    Johnson
7   Phoenix Suns    F       Ish  Wainright
8   Phoenix Suns  F-C     Dario      Saric
9   Phoenix Suns  F-C     Frank   Kaminsky
10  Phoenix Suns    G     Chris       Paul
11  Phoenix Suns    G    Elfrid     Payton
12  Phoenix Suns    G     Devin     Booker
13  Phoenix Suns    G   Cameron      Payne
14  Phoenix Suns    G     Aaron    Holiday
15  Phoenix Suns    G    Landry     Shamet
16  Phoenix Suns    G   Gabriel   Lundberg
```

In [9]:
find_roster_of_phx_suns =\
"""
-- BEGIN SOLUTION
SELECT teams.fullName AS teamName,
       players.pos,
       players.firstName,
       players.lastName
  FROM players
  JOIN teams
    ON players.teamId = teams.teamId
 WHERE teams.fullName = 'Phoenix Suns'
 ORDER BY players.pos;
-- END SOLUTION
"""

## 09. Write a SQL statement that is able to find the top 10 scorers with the most points in `players` and `career_summaries`.

```
  firstName   lastName  points
0    LeBron      James   36985
1   Carmelo    Anthony   28223
2     Kevin     Durant   25213
3     James     Harden   23281
4   Russell  Westbrook   23140
5     Chris       Paul   20844
6  LaMarcus   Aldridge   20558
7   Stephen      Curry   20064
8     DeMar    DeRozan   19610
9    Dwight     Howard   19377
```

In [10]:
find_the_top_ten_scorers =\
"""
-- BEGIN SOLUTION
SELECT players.firstName,
       players.lastName,
       career_summaries.points
  FROM players
  JOIN career_summaries
    ON players.personId = career_summaries.personId
 ORDER BY points DESC
 LIMIT 10;
-- END SOLUTION
"""

## 10. Write a SQL statement that is able to find the players with the most `ppg`, `rpg`, `apg`, `bpg`, and `spg` in `career_summaries` and `players`.

```
  firstName  lastName  key  value
0     Andre  Drummond  rpg   13.3
1   Anthony     Davis  bpg    2.3
2     Chris      Paul  apg    9.5
3     Chris      Paul  spg    2.1
4     Kevin    Durant  ppg   27.1
5    LeBron     James  ppg   27.1
6     Myles    Turner  bpg    2.3
```

In [11]:
find_the_top_players =\
"""
-- BEGIN SOLUTION
SELECT players.firstName,
       players.lastName,
       'ppg' AS key,
       career_summaries.ppg AS value
  FROM players
  JOIN career_summaries
    ON players.personId = career_summaries.personId
 WHERE players.personId IN (
           SELECT personId
             FROM career_summaries
            WHERE ppg = (
                SELECT MAX(ppg)
                  FROM career_summaries
            )
       )
 UNION
SELECT players.firstName,
       players.lastName,
       'rpg' AS key,
       career_summaries.rpg AS value
  FROM players
  JOIN career_summaries
    ON players.personId = career_summaries.personId
 WHERE players.personId IN (
           SELECT personId
             FROM career_summaries
            WHERE rpg = (
                SELECT MAX(rpg)
                  FROM career_summaries
            )
       )
 UNION
SELECT players.firstName,
       players.lastName,
       'apg' AS key,
       career_summaries.apg AS value
  FROM players
  JOIN career_summaries
    ON players.personId = career_summaries.personId
 WHERE players.personId IN (
           SELECT personId
             FROM career_summaries
            WHERE apg = (
                SELECT MAX(apg)
                  FROM career_summaries
            )
       )
 UNION
SELECT players.firstName,
       players.lastName,
       'bpg' AS key,
       career_summaries.bpg AS value
  FROM players
  JOIN career_summaries
    ON players.personId = career_summaries.personId
 WHERE players.personId IN (
           SELECT personId
             FROM career_summaries
            WHERE bpg = (
                SELECT MAX(bpg)
                  FROM career_summaries
            )
       )
 UNION
SELECT players.firstName,
       players.lastName,
       'spg' AS key,
       career_summaries.spg AS value
  FROM players
  JOIN career_summaries
    ON players.personId = career_summaries.personId
 WHERE players.personId IN (
           SELECT personId
             FROM career_summaries
            WHERE spg = (
                SELECT MAX(spg)
                  FROM career_summaries
            )
       );
-- END SOLUTION
"""

## End of assignment, run the following cells to get test result.

In [13]:
class TestAssignmentThree(unittest.TestCase):
    def test_01_retrieve_table_list_from_nbadb(self):
        table_list_from_nbadb = pd.read_sql(retrieve_table_list_from_nbadb, connection)
        self.assertEqual(table_list_from_nbadb.shape, (3, 2))
        types = table_list_from_nbadb.iloc[:, 0].values
        names = table_list_from_nbadb.iloc[:, 1].values
        self.assertIn("table", types)
        self.assertIn("career_summaries", names)
        self.assertIn("players", names)
        self.assertIn("teams", names)
    def test_02_retrieve_table_info_from_nbadb(self):
        table_info_from_nbadb = pd.read_sql(retrieve_table_info_from_nbadb, connection)
        self.assertEqual(table_info_from_nbadb.shape, (3, 3))
        names = table_info_from_nbadb.iloc[:, 0].values
        self.assertIn("career_summaries", names)
        self.assertIn("players", names)
        self.assertIn("teams", names)
        number_of_rows = table_info_from_nbadb.iloc[:, 1].values
        self.assertIn(502, number_of_rows)
        self.assertIn(30, number_of_rows)
        number_of_columns = table_info_from_nbadb.iloc[:, 2].values
        self.assertIn(31, number_of_columns)
        self.assertIn(18, number_of_columns)
        self.assertIn(12, number_of_columns)
    def test_03_count_players_from_each_country(self):
        players_from_each_country = pd.read_sql(count_players_from_each_country, connection)
        self.assertEqual(players_from_each_country.shape, (42, 2))
        countries = players_from_each_country.iloc[:, 0].values
        self.assertIn("USA", countries)
        self.assertIn("Canada", countries)
        self.assertIn("France", countries)
        self.assertIn("Ukraine", countries)
        self.assertIn("Portugal", countries)
        number_of_players = players_from_each_country.iloc[:, 1].values
        self.assertEqual(number_of_players.sum(), 502)
    def test_04_count_players_from_usa_or_not(self):
        players_from_usa_or_not = pd.read_sql(count_players_from_usa_or_not, connection)
        self.assertEqual(players_from_usa_or_not.shape, (2, 2))
        from_usa_or_not = players_from_usa_or_not.iloc[:, 0].values
        self.assertEqual(np.unique(from_usa_or_not).size, 2)
        number_of_players = players_from_usa_or_not.iloc[:, 1].values
        self.assertEqual(number_of_players.sum(), 502)
    def test_05_count_players_for_each_position(self):
        players_for_each_position = pd.read_sql(count_players_for_each_position, connection)
        self.assertEqual(players_for_each_position.shape, (7, 2))
        positions = players_for_each_position.iloc[:, 0].values
        self.assertIn("G", positions)
        self.assertIn("F", positions)
        self.assertIn("G-F", positions)
        self.assertIn("F-C", positions)
        self.assertIn("C", positions)
        self.assertIn("C-F", positions)
        self.assertIn("F-G", positions)
        number_of_players = players_for_each_position.iloc[:, 1].values
        self.assertEqual(number_of_players.sum(), 502)
    def test_06_count_players_for_each_new_position(self):
        players_for_each_new_position = pd.read_sql(count_players_for_each_new_position, connection)
        self.assertEqual(players_for_each_new_position.shape, (3, 2))
        positions = players_for_each_new_position.iloc[:, 0].values
        self.assertIn("G", positions)
        self.assertIn("F", positions)
        self.assertIn("C", positions)
        number_of_players = players_for_each_new_position.iloc[:, 1].values
        self.assertEqual(number_of_players.sum(), 502)
    def test_07_calculate_proportion_for_each_position(self):
        proportion_for_each_position = pd.read_sql(calculate_proportion_for_each_position, connection)
        self.assertEqual(proportion_for_each_position.shape, (7, 2))
        positions = proportion_for_each_position.iloc[:, 0].values
        self.assertIn("G", positions)
        self.assertIn("F", positions)
        self.assertIn("G-F", positions)
        self.assertIn("F-C", positions)
        self.assertIn("C", positions)
        self.assertIn("C-F", positions)
        self.assertIn("F-G", positions)
    def test_08_find_roster_of_phx_suns(self):
        roster_of_phx_suns = pd.read_sql(find_roster_of_phx_suns, connection)
        self.assertEqual(roster_of_phx_suns.shape, (17, 4))
        team_names = roster_of_phx_suns.iloc[:, 0].values
        self.assertIn("Phoenix Suns", team_names)
        positions = roster_of_phx_suns.iloc[:, 1].values
        self.assertIn("C", positions)
        self.assertIn("C-F", positions)
        self.assertIn("F", positions)
        self.assertIn("F-C", positions)
        self.assertIn("G", positions)
        last_names = roster_of_phx_suns.iloc[:, 3].values
        self.assertIn("Ayton", last_names)
        self.assertIn("Crowder", last_names)
        self.assertIn("Bridges", last_names)
        self.assertIn("Paul", last_names)
        self.assertIn("Booker", last_names)
    def test_09_find_the_top_ten_scorers(self):
        the_top_ten_scorers = pd.read_sql(find_the_top_ten_scorers, connection)
        self.assertEqual(the_top_ten_scorers.shape, (10, 3))
        first_names = the_top_ten_scorers.iloc[:, 0].values
        self.assertIn("LeBron", first_names)
        self.assertIn("Carmelo", first_names)
        self.assertIn("Kevin", first_names)
        last_names = the_top_ten_scorers.iloc[:, 1].values
        self.assertIn("James", last_names)
        self.assertIn("Anthony", last_names)
        self.assertIn("Durant", last_names)
    def test_10_find_the_top_players(self):
        the_top_players = pd.read_sql(find_the_top_players, connection)
        self.assertEqual(the_top_players.shape, (7, 4))
        first_names = the_top_players.iloc[:, 0].values
        self.assertIn("Andre", first_names)
        self.assertIn("Anthony", first_names)
        self.assertIn("Chris", first_names)
        self.assertIn("LeBron", first_names)
        last_names = the_top_players.iloc[:, 1].values
        self.assertIn("Drummond", last_names)
        self.assertIn("Davis", last_names)
        self.assertIn("Paul", last_names)
        self.assertIn("James", last_names)
        keys = the_top_players.iloc[:, 2].values
        self.assertIn("rpg", keys)
        self.assertIn("bpg", keys)
        self.assertIn("apg", keys)
        self.assertIn("spg", keys)
        self.assertIn("ppg", keys)

connection = sqlite3.connect('nba.db')
suite = unittest.TestLoader().loadTestsFromTestCase(TestAssignmentThree)
runner = unittest.TextTestRunner(verbosity=2)
test_results = runner.run(suite)
number_of_failures = len(test_results.failures)
number_of_errors = len(test_results.errors)
number_of_test_runs = test_results.testsRun
number_of_successes = number_of_test_runs - (number_of_failures + number_of_errors)

test_01_retrieve_table_list_from_nbadb (__main__.TestAssignmentThree) ... ok
test_02_retrieve_table_info_from_nbadb (__main__.TestAssignmentThree) ... ok
test_03_count_players_from_each_country (__main__.TestAssignmentThree) ... ok
test_04_count_players_from_usa_or_not (__main__.TestAssignmentThree) ... ok
test_05_count_players_for_each_position (__main__.TestAssignmentThree) ... ok
test_06_count_players_for_each_new_position (__main__.TestAssignmentThree) ... ok
test_07_calculate_proportion_for_each_position (__main__.TestAssignmentThree) ... ok
test_08_find_roster_of_phx_suns (__main__.TestAssignmentThree) ... ok
test_09_find_the_top_ten_scorers (__main__.TestAssignmentThree) ... ok
test_10_find_the_top_players (__main__.TestAssignmentThree) ... ok

----------------------------------------------------------------------
Ran 10 tests in 0.083s

OK


In [14]:
print("You've got {} successes among {} questions.".format(number_of_successes, number_of_test_runs))

You've got 10 successes among 10 questions.
